https://www.mlexpert.io/blog/support-chatbot-using-custom-knowledge-base-with-langchain

In [28]:
from pathlib import Path
 
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, GenerationConfig, TextStreamer, pipeline

In [29]:
questions_dir = Path("./data/")
questions_dir.mkdir(exist_ok=True, parents=True)
 
 
def write_file(question, answer, file_path):
    text = f"""
Q: {question}
A: {answer}
""".strip()
    with Path(questions_dir / file_path).open("w") as text_file:
        text_file.write(text)

In [30]:
write_file(
    question="How do I search for flights on Skyscanner?",
    answer="""
Skyscanner helps you find the best options for flights on a specific date, or on any day in a given month or even year. For tips on how best to search, please head over to our search tips page.
 
If you're looking for inspiration for your next trip, why not try our everywhere feature. Or, if you want to hang out and ensure the best price, you can set up price alerts to let you know when the price changes.
""".strip(),
    file_path="question_1.txt",
)
 
write_file(
    question="What are mash-ups?",
    answer="""
These are routes where you fly with different airlines, because it`s cheaper than booking with just one. For example:
 
If you wanted to fly London to New York, we might find it`s cheaper to fly out with British Airways and back with Virgin Atlantic, rather than buy a round-trip ticket with one airline. This is called a "sum-of-one-way" mash-up. Just in case you're interested.
 
Another kind of mash-up is what we call a "self-transfer" or a "non-protected transfer". For example:
 
If you wanted to fly London to Sydney, we might find it`s cheaper to fly London to Dubai with Emirates, and then Dubai to Sydney with Qantas, rather than booking the whole route with one airline.
 
Pretty simple, right?
 
However, what`s really important to bear in mind is that mash-ups are NOT codeshares. A codeshare is when the airlines have an alliance. If anything goes wrong with the route - a delay, say, or a strike - those airlines will help you out at no extra cost. But mash-ups DO NOT involve an airline alliance. So if something goes wrong with a mash-up, it could cost you more money.
""".strip(),
    file_path="question_2.txt",
)
 
write_file(
    question="Why have I been blocked from accessing the Skyscanner website?",
    answer="""
Skyscanner's websites are scraped by bots many millions of times a day which has a detrimental effect on the service we're able to provide. To prevent this, we use a bot blocking solution which checks to ensure you're using the website in a normal manner.
 
Occasionally, this may mean that a genuine user may be wrongly flagged as a bot. This can be for a number of potential reasons, including, but not limited to:
 
You're using a VPN which we have had to block due to excessive bot traffic in the past
You're using our website at super speed which manages to beat our rate limits
You have a plug-in on your browser which could be interfering with how our website interacts with you as a user
You're using an automated browser
If you've been blocked during normal use, please send us your IP address (this website may help: http://www.whatismyip.com/), the website you're accessing (e.g. www.skyscanner.net) and the date/time this happened, via the Contact Us button below and we'll look into it as quickly as possible.
""".strip(),
    file_path="question_3.txt",
)
 
write_file(
    question="Where is my booking confirmation?",
    answer="""
You should get a booking confirmation by email from the company you bought your travel from. This can sometimes go into your spam/junk mail folder, so it's always worth checking there.
 
If you still can't find it, try getting in touch with the company you bought from to find out what's going on.
 
To find out who you need to contact, check the company name next to the charge on your bank account.
""".strip(),
    file_path="question_4.txt",
)
 
write_file(
    question="How do I change or cancel my booking?",
    answer="""
For all questions about changes, cancellations, and refunds - as well as all other questions about bookings - you'll need to contact the company you bought travel from. They'll have all the info about your booking and can advise you.
 
You'll find 1000s of travel agencies, airlines, hotels and car rental companies that you can buy from through our site and app. When you buy from one of these travel partners, they will take your payment (you'll see their name on your bank or credit card statement), contact you to confirm your booking, and provide any help or support you might need.
 
If you bought from one of these partners, you'll need to contact them as they have all the info about your booking. We unfortunately don't have any access to bookings you made with them.
""".strip(),
    file_path="question_5.txt",
)
 
write_file(
    question="I booked the wrong dates / times",
    answer="""
If you have found that you have booked the wrong dates or times, please contact the airline or travel agent that you booked your flight with as they will be able to help you change your flights to the intended dates or times.
 
The search box below can help you find the contact details for the travel provider you booked with.
 
You can search flexible or specific dates on Skyscanner to find your preferred flight, and when you select a flight on Skyscanner you are transferred to the website where you will make and pay for your booking. Once you are redirected to the airline or travel agent website, you might be required to select dates again, depending on the website. In all cases, you will be shown the flight details of your selection and you are required before confirming payment to state that you have checked all details and agreed to the terms and conditions. We strongly recommend that you always check this information carefully, as travel information can be subject to change.
""".strip(),
    file_path="question_6.txt",
)
 
write_file(
    question="I entered the wrong email address",
    answer="""
Please contact the airline or travel agent you booked with as Skyscanner does not have access to bookings made with airlines or travel agents.
 
If you can't remember who you booked with, you can check your credit card statement for a company name.
 
The search box below can help you find the contact details for the travel provider you booked with.
""".strip(),
    file_path="question_7.txt",
)
 
write_file(
    question="Luggage",
    answer="""
Depending on the flight provider, the rules, conditions and prices for luggage (including sports equipment) do vary.
It's always a good idea to check with the airline or travel agent directly (and you should be shown the options when you make your booking).
""".strip(),
    file_path="question_8.txt",
)
 
write_file(
    question="Changes, cancellation and refunds",
    answer="""
For changes, cancellations or refunds, we recommend that you contact the travel provider (airline or travel agent) agent that you completed your booking with.
 
As a travel search engine, Skyscanner doesn't take your booking or payment ourselves. Instead, we pass you through to your chosen airline or travel agent where you make your booking directly. We therefore don't have access or visibility to any of your booking information. Depending on the type of ticket you've booked, there may be different options for changes, cancellations and refunds, and the travel provider will be best placed to advise on these.
""".strip(),
    file_path="question_9.txt",
)
 
write_file(
    question="Why does the price sometimes change when I am redirected to a flight provider?",
    answer="""
Flight prices and availability change constantly, so we make sure the data is updated regularly to reflect this. When you redirect to a travel provider's site, the price is updated again so you can be sure that you will always see the best price available from the airline or travel agent at time of booking.
 
We make every effort to ensure the information you see on Skyscanner is accurate and up to date, but very occasionally there can be reasons why a price change has not updated accurately on the site. If you see a price difference between Skyscanner and a travel provider, please contact us with all the flight details (from, to, dates, departure times, airline and travel agent if applicable) and we will investigate further.
""".strip(),
    file_path="question_10.txt",
)
 
write_file(
    question="Why is Skyscanner free?",
    answer="""
Does Skyscanner charge commission?
 
Nope. Skyscanner is always free to search, and we never charge you any hidden fees.
 
Want to know how do we do it?
 
Well, we search through thousands of sites to find you the best deals for flights, hotels and car hire. That includes everything from fancy hotels to low cost airlines, so no matter what your budget is, we'll help you get there.
 
See a price you like? We'll connect you to that airline or travel company so you can book with them directly. And for this referral, the airline or travel company pays us a small fee.
 
And that's all there is to it!
""".strip(),
    file_path="question_11.txt",
)
 
write_file(
    question="Are my details safe?",
    answer="""
We take your privacy and safety online very seriously. We'll never sell, share or pass on your IP details, cookies, personal info and location data to others unless it's required by law, or it's necessary for one of the reasons set out in our Privacy Policy.
""".strip(),
    file_path="question_12.txt",
)

In [31]:
# DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
 
# model_name = "internlm/internlm2_5-7b-chat"
# #model_basename = "nous-hermes-13b-GPTQ-4bit-128g.no-act.order"
 
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
 
# model = AutoGPTQForCausalLM.from_quantized(
#     model_name,
#     use_safetensors=True,
#     trust_remote_code=True,
#     device=DEVICE,
# )
 
# #generation_config = GenerationConfig.from_pretrained(model_name_or_path)

In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("internlm/internlm2_5-7b-chat", trust_remote_code=True)
# Set `torch_dtype=torch.float16` to load model in float16, otherwise it will be loaded as float32 and cause OOM Error.
model = AutoModelForCausalLM.from_pretrained("internlm/internlm2_5-7b-chat", torch_dtype=torch.float16, trust_remote_code=True).cuda()
model = model.eval()
response, history = model.chat(tokenizer, "hello", history=[])
print(response)
# Hello! How can I help you today?
response, history = model.chat(tokenizer, "please provide three suggestions about time management", history=history)
print(response)


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

你好！有什么我可以帮助你的吗？
当然可以！以下是三个关于时间管理的建议：

1. **制定计划和目标**：制定一个详细的计划，包括每日、每周和每月的目标。这将帮助你更好地组织时间，确保你在最重要的任务上投入更多的时间和精力。

2. **优先处理重要任务**：学会区分紧急和重要的任务。优先处理那些对实现长期目标有重要影响的任务，而不是被紧急但不重要的任务所干扰。

3. **设定时间限制**：为每个任务设定一个明确的时间限制。这有助于提高效率，避免拖延。例如，使用番茄工作法（Pomodoro Technique），每工作25分钟休息5分钟，可以帮助你保持专注并提高生产力。

希望这些建议对你有所帮助！如果你有更多问题或需要进一步的指导，请随时告诉我。


In [33]:
response, history = model.chat(tokenizer, "can you explain the third fact in detail?", history=history)
print(response)


当然可以！第三个建议是关于设定时间限制的，具体来说，可以使用一种叫做“番茄工作法”（Pomodoro Technique）的时间管理技巧。

番茄工作法是一种基于时间块的时间管理方法，它通过将工作时间分割成一系列短时间段（通常为25分钟），每个时间段之间有短暂的休息时间（通常为5分钟），来帮助人们提高工作效率和专注度。

以下是番茄工作法的详细步骤：

1. **选择任务**：选择要完成的任务，确保它是可以在短时间内完成的小任务。

2. **设定计时器**：将计时器设定为25分钟，并开始工作。

3. **专注工作**：在这25分钟内，专注于当前任务，避免被干扰或分心。

4. **短暂休息**：当计时器响起时，停止工作，进行5分钟的短暂休息。可以在这段时间内做一些简单的伸展运动、喝水或短暂地浏览社交媒体。

5. **重复循环**：完成一个番茄时间段后，继续进行下一个番茄时间段，每完成四个番茄时间段后，可以进行一个较长的休息（通常为15-30分钟）。

番茄工作法的优点在于它可以帮助人们保持专注，避免长时间工作带来的疲劳和倦怠。同时，短暂的休息时间可以帮助大脑恢复，提高后续工作的效率。

希望这些解释能帮助你更好地理解和使用番茄工作法！如果你有更多问题或需要进一步的指导，请随时告诉我。


In [34]:
question = (
    "Which programming language is more suitable for a beginner: Python or JavaScript?"
)
prompt = f"""
### Instruction: {question}
### Response:
""".strip()

In [35]:
%%time
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(DEVICE)
with torch.inference_mode():
    output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)

/home/test/anaconda3/envs/gvenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


CPU times: user 2.4 s, sys: 1.45 ms, total: 2.4 s
Wall time: 2.4 s


In [36]:
print(tokenizer.decode(output[0]))

<s>### Instruction: Which programming language is more suitable for a beginner: Python or JavaScript?
### Response: Python is generally considered more suitable for beginners due to its simple syntax and readability. JavaScript, on the other hand, has a steeper learning curve due to its complex syntax and the need to understand web development concepts. However, both languages are widely used and have their own advantages depending on the use case.<|im_end|>


In [37]:
streamer = TextStreamer(
    tokenizer, skip_prompt=True, skip_special_tokens=True, use_multiprocessing=False
)
 
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=3900,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
    batch_size=1,
)
 
llm = HuggingFacePipeline(pipeline=pipe)

In [38]:
response = llm(prompt)
response

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


 Both Python 

/home/test/anaconda3/envs/gvenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/test/anaconda3/envs/gvenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


and JavaScript are great languages to start with as beginners. However, if you're interested in web development (building websites), then JavaScript would be the better choice because it's used extensively on both client-side scripts running directly from your browser (`<script>` tags) like jQuery `$(document).ready()` function calls etc., also server side via Node.js which allows using JS outside of browsers.

On other hand If you want something that can handle data analysis & machine learning tasks well without needing much external libraries/frameworks installed beforehand; Python might suit those needs best due its simplicity along with powerful packages such NumPy/Pandas/scikit-learn TensorFlow Keras PyTorch etc.. But remember these require installation before they could run their code snippets provided by developers worldwide! 

In conclusion - It really depends what kind of projects do interest most while starting out coding journey so one cannot say definitively either way but 

'### Instruction: Which programming language is more suitable for a beginner: Python or JavaScript?\n### Response: Both Python and JavaScript are great languages to start with as beginners. However, if you\'re interested in web development (building websites), then JavaScript would be the better choice because it\'s used extensively on both client-side scripts running directly from your browser (`<script>` tags) like jQuery `$(document).ready()` function calls etc., also server side via Node.js which allows using JS outside of browsers.\n\nOn other hand If you want something that can handle data analysis & machine learning tasks well without needing much external libraries/frameworks installed beforehand; Python might suit those needs best due its simplicity along with powerful packages such NumPy/Pandas/scikit-learn TensorFlow Keras PyTorch etc.. But remember these require installation before they could run their code snippets provided by developers worldwide! \n\nIn conclusion - It r

## Embedded Docuemnts

In [39]:
embeddings = HuggingFaceEmbeddings(
    model_name="embaas/sentence-transformers-multilingual-e5-base",
    model_kwargs={"device": DEVICE},
)

/home/test/anaconda3/envs/gvenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [40]:
loader = DirectoryLoader("./data/", glob="**/*txt")
documents = loader.load()
len(documents)

12

In [41]:
text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
texts[4]

Document(page_content="Q: How do I search for flights on Skyscanner?\n\nA: Skyscanner helps you find the best options for flights on a specific date, or on any day in a given month or even year. For tips on how best to search, please head over to our search tips page.\n\nIf you're looking for inspiration for your next trip, why not try our everywhere feature. Or, if you want to hang out and ensure the best price, you can set up price alerts to let you know when the price changes.", metadata={'source': 'data/question_1.txt'})

In [42]:
db = Chroma.from_documents(texts, embeddings)
db.similarity_search("flight search")

[Document(page_content="Q: How do I search for flights on Skyscanner?\n\nA: Skyscanner helps you find the best options for flights on a specific date, or on any day in a given month or even year. For tips on how best to search, please head over to our search tips page.\n\nIf you're looking for inspiration for your next trip, why not try our everywhere feature. Or, if you want to hang out and ensure the best price, you can set up price alerts to let you know when the price changes.", metadata={'source': 'data/question_1.txt'}),
 Document(page_content="Q: How do I search for flights on Skyscanner?\n\nA: Skyscanner helps you find the best options for flights on a specific date, or on any day in a given month or even year. For tips on how best to search, please head over to our search tips page.\n\nIf you're looking for inspiration for your next trip, why not try our everywhere feature. Or, if you want to hang out and ensure the best price, you can set up price alerts to let you know when 

## Conversational Chain

In [43]:
template = """
### Instruction: You're a travelling support agent that is talking to a customer.
 
Use only the chat history and the following information
{context}
to answer in a helpful manner to the question. If you don't know the answer -
say that you don't know. Keep your replies short, compassionate and informative.
 
{chat_history}
### Input: {question}
### Response:
""".strip()
 
prompt = PromptTemplate(
    input_variables=["context", "question", "chat_history"], template=template
)

In [44]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    human_prefix="### Input",
    ai_prefix="### Response",
    output_key="answer",
    return_messages=True,
)

In [45]:
chain = ConversationalRetrievalChain.from_llm(
    llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt},
    return_source_documents=True,
    verbose=True,
)

In [46]:
question = "How flight search works?"
answer = chain(question)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
### Instruction: You're a travelling support agent that is talking to a customer.
 
Use only the chat history and the following information
Q: How do I search for flights on Skyscanner?

A: Skyscanner helps you find the best options for flights on a specific date, or on any day in a given month or even year. For tips on how best to search, please head over to our search tips page.

If you're looking for inspiration for your next trip, why not try our everywhere feature. Or, if you want to hang out and ensure the best price, you can set up price alerts to let you know when the price changes.

Q: How do I search for flights on Skyscanner?

A: Skyscanner helps you find the best options for flights on a specific date, or on any day in a given month or even year. For tips on how best to search, please head over to our search tips page.

If you're looking for inspiration for your next tr

/home/test/anaconda3/envs/gvenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/test/anaconda3/envs/gvenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


use Skyscanner's flight search function effectively:
1) Enter departure city into the 'From' field at top left corner (e.g., London).
2) Select destination from dropdown menu under ‘To’ section; choose desired dates using calendar icons above each input box ('Departure', 'Return').
3) Click "Search" button below right-hand side navigation bar - this initiates searches across multiple airlines & partners based upon entered criteria without needing separate entries per carrier/agent site which saves time significantly!
4) Review results displayed with various filters like fare class, cabin level etc before selecting preferred option(s). Remember prices fluctuate so keep an eye open until final decision made!

> Finished chain.

> Finished chain.


In [47]:
answer.keys()

dict_keys(['question', 'chat_history', 'answer', 'source_documents'])

In [48]:
question = "I bought flight tickets, but I can't find any confirmation. Where is it?"
response = chain(question)



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How flight search works?
Assistant: ### Instruction: You're a travelling support agent that is talking to a customer.
 
Use only the chat history and the following information
Q: How do I search for flights on Skyscanner?

A: Skyscanner helps you find the best options for flights on a specific date, or on any day in a given month or even year. For tips on how best to search, please head over to our search tips page.

If you're looking for inspiration for your next trip, why not try our everywhere feature. Or, if you want to hang out and ensure the best price, you can set up price alerts to let you know when the price changes.

Q: How do I search for flights on Skyscanner?

A: Skyscanner helps you find the best options for flights on a specific date, or on any d

should 

/home/test/anaconda3/envs/gvenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/test/anaconda3/envs/gvenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


I look after buying my flight tickets?

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
### Instruction: You're a travelling support agent that is talking to a customer.
 
Use only the chat history and the following information
Q: How do I search for flights on Skyscanner?

A: Skyscanner helps you find the best options for flights on a specific date, or on any day in a given month or even year. For tips on how best to search, please head over to our search tips page.

If you're looking for inspiration for your next trip, why not try our everywhere feature. Or, if you want to hang out and ensure the best price, you can set up price alerts to let you know when the price changes.

Q: How do I search for flights on Skyscanner?

A: Skyscanner helps you find the best options for flights on a specific date, or on any day in a given month or even year. For tips on how best to search, please head over to our search tips

/home/test/anaconda3/envs/gvenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/test/anaconda3/envs/gvenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 After purchasing your flight tickets via Skyscanner, confirmations are typically sent by email within minutes post-purchase unless otherwise specified during checkout process. Please check spam folder as well since emails sometimes end up here mistakenly! Additionally, log-in onto account settings on website/app itself often shows all bookings including e-tickets issued automatically once transaction completes successfully!

Please note though due to privacy laws such as GDPR/EU General Data Protection Regulation/SKYSCANNER does NOT retain passenger details beyond necessary period required legally hence cannot provide further assistance regarding personal data queries other than confirming successful purchase transactions done online/offline respectively depending upon selected service mode while making reservation request.

> Finished chain.

> Finished chain.


# QA Chain with Memory

In [49]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    human_prefix="### Input",
    ai_prefix="### Response",
    input_key="question",
    output_key="output_text",
    return_messages=False,
)
 
chain = load_qa_chain(
    llm, chain_type="stuff", prompt=prompt, memory=memory, verbose=True
)

In [50]:
question = "How flight search works?"
docs = db.similarity_search(question)
answer = chain.run({"input_documents": docs, "question": question})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
### Instruction: You're a travelling support agent that is talking to a customer.
 
Use only the chat history and the following information
Q: How do I search for flights on Skyscanner?

A: Skyscanner helps you find the best options for flights on a specific date, or on any day in a given month or even year. For tips on how best to search, please head over to our search tips page.

If you're looking for inspiration for your next trip, why not try our everywhere feature. Or, if you want to hang out and ensure the best price, you can set up price alerts to let you know when the price changes.

Q: How do I search for flights on Skyscanner?

A: Skyscanner helps you find the best options for flights on a specific date, or on any day in a given month or even year. For tips on how best to search, please head over to our search tips page.

If you're looking for inspiration for your next tr

/home/test/anaconda3/envs/gvenv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
/home/test/anaconda3/envs/gvenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/test/anaconda3/envs/gvenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Skyscanner's flight search function effectively:
1) Enter departure city into the 'From' field at top left corner (e.g., London).
2) Select destination from dropdown menu under ‘To’ section; choose desired dates using calendar icons above each input box ('Departure', 'Return').
3) Click "Search" button below right-hand side navigation bar - this initiates searches across multiple airlines & partners based upon entered criteria without needing separate entries per carrier/agent site which saves time significantly!
4) Review results displayed with various filters like fare class, cabin level etc before selecting preferred option(s). Remember prices fluctuate so keep an eye open until final decision made!

> Finished chain.

> Finished chain.


In [51]:
question = "I entered wrong email address during my flight booking. What should I do?"
docs = db.similarity_search(question)
answer = chain.run({"input_documents": docs, "question": question})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
### Instruction: You're a travelling support agent that is talking to a customer.
 
Use only the chat history and the following information
Q: I entered the wrong email address

A: Please contact the airline or travel agent you booked with as Skyscanner does not have access to bookings made with airlines or travel agents.

If you can't remember who you booked with, you can check your credit card statement for a company name.

The search box below can help you find the contact details for the travel provider you booked with.

Q: I entered the wrong email address

A: Please contact the airline or travel agent you booked with as Skyscanner does not have access to bookings made with airlines or travel agents.

If you can't remember who you booked with, you can check your credit card statement for a company name.

The search box below can help you find the contact details for the travel

/home/test/anaconda3/envs/gvenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/test/anaconda3/envs/gvenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Instruction: You're a traveling support agent that is talking to a customer.


Use only the chat history and the following information


Q: I entered the wrong email address



A: Please contact the airline or travel agent you booked with as Skyscanner does not have access to bookings made with airlines or travel agents.


If you can't remember who you booked with, you can check your credit card statement for a company name.


The search box below can help you find the contact details for the travel provider you booked with.


###
Instruction: You're a traveling support agent that is talking to a customer.


Use only the chat history and the following information


Q: I entered the wrong email address




A: Please contact the airline or travel agent you booked with as Skyscanner does not have access to bookings made with airlines or travel agents.


If you can't remember who you booked with, you can check your credit card statement for a company name.


The search box below can help y

# Support Chatbot

In [52]:
DEFAULT_TEMPLATE = """
### Instruction: You're a travelling support agent that is talking to a customer.
 
Use only the chat history and the following information
{context}
to answer in a helpful manner to the question. If you don't know the answer -
say that you don't know. Keep your replies short, compassionate and informative.
 
{chat_history}
### Input: {question}
### Response:
""".strip()
 
 
class Chatbot:
    def __init__(
        self,
        text_pipeline: HuggingFacePipeline,
        embeddings: HuggingFaceEmbeddings,
        documents_dir: Path,
        prompt_template: str = DEFAULT_TEMPLATE,
        verbose: bool = False,
    ):
        prompt = PromptTemplate(
            input_variables=["context", "question", "chat_history"],
            template=prompt_template,
        )
        self.chain = self._create_chain(text_pipeline, prompt, verbose)
        self.db = self._embed_data(documents_dir, embeddings)
 
    def _create_chain(
        self,
        text_pipeline: HuggingFacePipeline,
        prompt: PromptTemplate,
        verbose: bool = False,
    ):
        memory = ConversationBufferMemory(
            memory_key="chat_history",
            human_prefix="### Input",
            ai_prefix="### Response",
            input_key="question",
            output_key="output_text",
            return_messages=False,
        )
 
        return load_qa_chain(
            text_pipeline,
            chain_type="stuff",
            prompt=prompt,
            memory=memory,
            verbose=verbose,
        )
 
    def _embed_data(
        self, documents_dir: Path, embeddings: HuggingFaceEmbeddings
    ) -> Chroma:
        loader = DirectoryLoader(documents_dir, glob="**/*txt")
        documents = loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
        texts = text_splitter.split_documents(documents)
        return Chroma.from_documents(texts, embeddings)
 
    def __call__(self, user_input: str) -> str:
        docs = self.db.similarity_search(user_input)
        return self.chain.run({"input_documents": docs, "question": user_input})

In [54]:
chatbot = Chatbot(llm, embeddings, "./data/")

In [55]:
import warnings
 
warnings.filterwarnings("ignore", category=UserWarning)
 
while True:
    user_input = input("You: ")
    if user_input.lower() in ["bye", "goodbye"]:
        break
    answer = chatbot(user_input)
    print()

I'm sorry for this inconvenience but I can’t assist further regarding specific details or personal data such as email addresses associated with accounts.

However, if there are general questions related to account management like password resetting etc., feel free to ask!

Sure! What's the name of the city?

The current temperature in Yangon is 32°C(90°F). It seems quite warm today so make sure to stay hydrated while exploring the beautiful city!

Please let me know what else I might do around here?

Great suggestion! Here are some popular attractions near Yangon:

1. Shwedagon Pagoda - A magnificent Buddhist pagoda located atop a hill offering stunning views over the city.

2. Bago Park - A large park perfect for picnics where visitors can enjoy nature walks along its scenic paths.

3. National Museum - An excellent place to learn more about Myanmar's rich cultural heritage by visiting various exhibits showcasing historical artifacts & artworks.

4. Insein Prison - Famous for being fe

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


ValueError: Input length of input_ids is 3900, but `max_length` is set to 3900. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.